In [19]:
from tensorflow.python.client import device_lib
if __name__ == "__main__":
    print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16190921534383688794
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 18116442409695186861
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 12996118769601755625
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15616019008
locality {
  bus_id: 2
  numa_node: 1
  links {
  }
}
incarnation: 14614628884015053704
physical_device_desc: "device: 0, name: Tesla V100-PCIE-16GB, pci bus id: 0000:af:00.0, compute capability: 7.0"
]


In [6]:
!nvidia-smi

Failed to initialize NVML: Driver/library version mismatch


In [5]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

RuntimeError: CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory

In [1]:
import tensorflow as tf
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
tf.config.get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [13]:
tf.config.threading.get_inter_op_parallelism_threads(),tf.config.threading.get_intra_op_parallelism_threads()

(0, 0)

Disable GPU

In [15]:
tf.debugging.set_log_device_placement(True)
# tf.config.set_visible_devices([], 'GPU') #管用

# Place tensors on the CPU
# with tf.device('CPU:0'): #不管用
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])

c = tf.matmul(a, b)
print(c)

tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [14]:
import tensorflow as tf
from keras import backend as K

num_cores = 32

tf.config.threading.set_intra_op_parallelism_threads(num_cores)
tf.config.threading.set_inter_op_parallelism_threads(num_cores)

config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=num_cores,\
        inter_op_parallelism_threads=num_cores, allow_soft_placement=True,\
        device_count = {'CPU' : 1, 'GPU' : 0})
session = tf.Session(config=config)
K.set_session(session)

# 多线程加速

In [2]:
import multiprocessing
pool = multiprocessing.Pool()

In [3]:
multiprocessing.cpu_count()

32

In [ ]:
#Method 1 
#基于进程的并行
def task(pid):
    results = pid * pid * pid 
    return result

def main():
#     multiprocessing.freeze_support() #windowns
    pool = multiprocessing.Pool()
    cpus = multiprocessing.cpu_count()
    results = []

    for i in xrange(0, cpus):
        result = pool.apply_async(task, args=(i,))
        results.append(result)

    pool.close()
    pool.join()

    for result in results:
        print(result.get())

In [366]:
#Method 2
import multiprocessing as mp
import numpy as np
print('CPU numbers:',mp.cpu_count())
def _apply_df(args):
    df, func, kwargs = args
    return df.apply(func, **kwargs)

def apply_by_multiprocessing(df, func, **kwargs):
#     print(kwargs)
    workers = kwargs.pop('workers')
    pool = mp.Pool(processes=workers)
    result = pool.map(_apply_df, [(d, func, kwargs) for d in np.array_split(df, workers)])
    pool.close()
    return pd.concat(list(result))
#apply_by_multiprocessing(fullset['Text'], process_text, workers=cores)

CPU numbers: 32


In [361]:
import pandas as pd
df = pd.DataFrame(range(64*1000),columns=['col1'])
df['col2'] = range(64*1000,128*1000)
def getPos(a,b=2):
    c = (a^b) + (a/(b+1))+ (a*b)
    c = len((str(c)+str([i for i in range(100)])).split('1'))
    return c
    
apply_by_multiprocessing(df['col1'], getPos, workers=2,b=2)

0        21
1        21
2        21
3        21
4        22
         ..
63995    23
63996    22
63997    22
63998    22
63999    22
Name: col1, Length: 64000, dtype: int64

In [372]:
# import time
s_py = time.time()
s_cpu = time.clock()

# df['col1'].apply(getPos)
apply_by_multiprocessing(df['col1'], getPos, workers=16,b=2)
    

print("programme:",time.time()-s_py)
print("cpu:",time.clock()-s_cpu)

programme: 0.34027838706970215
cpu: 0.18056300000000647


In [391]:
df = pd.DataFrame(range(4),columns=['col1'])
for d in np.array_split(df,2):
    print(type(d))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
